<a href="https://colab.research.google.com/github/poddubnyoleg/nondualbot/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import keras.backend as K
import sys

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
file = open('/content/drive/My Drive/nondualbot/i_am_that.txt', 'r') 
data = file.read() 

In [4]:
data[:100]

'That which permeates all, which nothing transcends and which, like the universal space around us, fi'

In [0]:
chars = sorted(list(set(data)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [6]:

n_chars = len(data)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  988168
Total Vocab:  55


For each chunk of data [seq_length] we take one-hot-encoded dataX array and dataY array (same but shifted +1)

In [0]:
seq_length = 100
overlapping_length = 30
dataX = []
dataY = []
cur_pos = 0
while cur_pos<n_chars-seq_length-1:
  seq_input = np_utils.to_categorical([char_to_int[char] for char in data[cur_pos:cur_pos + seq_length]], num_classes=n_vocab)
  seq_output = np_utils.to_categorical([char_to_int[char] for char in data[cur_pos + 1:cur_pos + seq_length + 1]], num_classes=n_vocab)
  dataX.append(seq_input)
  dataY.append(seq_output)

  cur_pos+=10

In [9]:
X = np.reshape(dataX, (len(dataX), seq_length, n_vocab))
print(X.shape)

(98807, 100, 55)


In [10]:
y = np.reshape(dataY, (len(dataY), seq_length, n_vocab))
print(y.shape)

(98807, 100, 55)


In [11]:
X.shape[1], X.shape[2], y.shape[1]

(100, 55, 100)

In [47]:
K.clear_session()

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(n_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 256)          319488    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 256)          525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
dense_1 (Dense)              (None, 100, 55)           14135     
Total params: 858,935
Trainable params: 858,935
Non-trainable params: 0
_________________________________________________________________


In [0]:
filepath="/content/drive/My Drive/nondualbot/best_current_weights_1.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
num_data = [char_to_int[char] for char in data]

In [0]:
batch_size = 128

def train_generator():
  while True:
    x_train = []
    y_train = []
    for i in range(batch_size):
      cur_pos = np.random.randint(low=0, high=n_chars-seq_length-1)
      x_train.append(np_utils.to_categorical(num_data[cur_pos:cur_pos + seq_length], num_classes=n_vocab))
      y_train.append(np_utils.to_categorical(num_data[cur_pos + 1:cur_pos + seq_length + 1], num_classes=n_vocab))
    
    yield np.reshape(x_train, (batch_size, seq_length, n_vocab)), np.reshape(y_train, (batch_size, seq_length, n_vocab))


In [33]:
model.fit_generator(train_generator(), steps_per_epoch=1000, epochs=50, verbose=1, callbacks=callbacks_list)


Epoch 1/50
1000/1000 [==============================] - 417s 417ms/step - loss: 1.1857

Epoch 00001: loss improved from 1.21428 to 1.18567, saving model to /content/drive/My Drive/nondualbot/best_current_weights_1.hdf5
Epoch 2/50
1000/1000 [==============================] - 414s 414ms/step - loss: 1.1489

Epoch 00002: loss improved from 1.18567 to 1.14887, saving model to /content/drive/My Drive/nondualbot/best_current_weights_1.hdf5
Epoch 3/50
1000/1000 [==============================] - 411s 411ms/step - loss: 1.1176

Epoch 00003: loss improved from 1.14887 to 1.11762, saving model to /content/drive/My Drive/nondualbot/best_current_weights_1.hdf5
Epoch 4/50
1000/1000 [==============================] - 410s 410ms/step - loss: 1.0830

Epoch 00004: loss improved from 1.11762 to 1.08302, saving model to /content/drive/My Drive/nondualbot/best_current_weights_1.hdf5
Epoch 5/50
1000/1000 [==============================] - 416s 416ms/step - loss: 1.0520

Epoch 00005: loss improved from 1.08

In [22]:
np.random.randint(low=0, high=10)

8

In [17]:
model.fit(X, y, epochs=100, batch_size=128, callbacks=callbacks_list)

Epoch 1/100
98807/98807 [==============================] - 321s 3ms/step - loss: 2.1147

Epoch 00001: loss improved from inf to 2.11473, saving model to /content/drive/My Drive/nondualbot/best_current_weights_1.hdf5
Epoch 2/100
98807/98807 [==============================] - 316s 3ms/step - loss: 1.4609

Epoch 00002: loss improved from 2.11473 to 1.46091, saving model to /content/drive/My Drive/nondualbot/best_current_weights_1.hdf5
Epoch 3/100
98807/98807 [==============================] - 317s 3ms/step - loss: 1.3013

Epoch 00003: loss improved from 1.46091 to 1.30130, saving model to /content/drive/My Drive/nondualbot/best_current_weights_1.hdf5
Epoch 4/100
98807/98807 [==============================] - 315s 3ms/step - loss: 1.2143

Epoch 00004: loss improved from 1.30130 to 1.21428, saving model to /content/drive/My Drive/nondualbot/best_current_weights_1.hdf5
Epoch 5/100
 3584/98807 [>.............................] - ETA: 5:06 - loss: 1.1823

KeyboardInterrupt: ignored

In [0]:
# load the network weights
filename = "/content/drive/My Drive/nondualbot/best_current_weights_1.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [40]:

# pick a random seed
#start = np.random.randint(0, len(dataX)-1)
#pattern = dataX[start]
#print(''.join([int_to_char[value] for value in pattern_flat]))
question = 'Fuck off stupid bot.'
pattern = np_utils.to_categorical([char_to_int[char] for char in question], num_classes=n_vocab)
pattern = np.append(np.zeros(shape=(max(seq_length-pattern.shape[0],0), n_vocab)), pattern, axis=0)

print(question)

# generate characters
for i in range(150):

  prediction = model.predict(np.reshape(pattern, (1, pattern.shape[0], pattern.shape[1])), verbose=0)[0][-1]
  
  index = np.random.choice(len(prediction), p=(prediction**2)/sum(prediction**2))#np.argmax(prediction)#
  result = int_to_char[index]
  sys.stdout.write(result)
  pattern = np.append(pattern, [np_utils.to_categorical(index,num_classes=n_vocab)], axis=0)
  pattern = pattern[1:len(pattern)]
  #print("\nDone.")

Fuck off stupid bot.
 As a matter of fact, it is not so important. I am a person and therefore it is not experienceable, that you are not a person. This is the universal l

In [24]:
model.predict(np.reshape(pattern, (1, pattern.shape[0], pattern.shape[1])), verbose=0)[0][-1]

array([9.9999940e-01, 5.4691748e-07, 4.8891195e-12, 1.0469446e-09,
       2.6797808e-10, 6.7335869e-22, 2.8095636e-26, 6.3552258e-29,
       7.0487436e-23, 6.3646487e-26, 3.1674805e-26, 4.9144446e-28,
       1.0800542e-21, 1.1273043e-25, 3.6994567e-28, 6.7637692e-34,
       3.3639891e-28, 2.0014950e-27, 2.9739306e-31, 5.9701945e-25,
       2.0236516e-27, 0.0000000e+00, 1.8087869e-34, 1.6505187e-24,
       1.3041807e-22, 2.8014332e-31, 8.8703745e-35, 1.8485967e-19,
       6.2777927e-26, 6.5178172e-19, 1.2295411e-16, 1.3757429e-19,
       6.6819938e-13, 3.0502608e-13, 2.4411573e-12, 2.3483241e-15,
       1.7099024e-17, 1.6124142e-16, 2.0943558e-22, 2.6491773e-17,
       1.4324105e-12, 8.1101988e-14, 7.6933120e-11, 5.4370323e-14,
       4.9228969e-17, 4.9811773e-23, 2.1248222e-15, 8.4897835e-12,
       2.5380966e-16, 1.9712886e-15, 8.8841031e-17, 3.3209028e-15,
       2.6880777e-22, 6.2216196e-20, 2.2106736e-28], dtype=float32)

In [58]:
np.append(pattern, [np_utils.to_categorical(index,num_classes=n_vocab)], axis=0)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [57]:
patte

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.], dtype=float32)

In [23]:
np.argmax(dataY[0])

40

In [24]:
dataY[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.], dtype=float32)